# STAC to GeoCroissant Conversion
<img src="../assets/GeoCroissant.jpg" alt="GeoCroissant" width="150" style="float: right; margin-left: 50px;">
This notebook converts metadata from a [STAC Collection](https://stacspec.org/en) into [GeoCroissant](https://github.com/mlcommons/croissant) JSON-LD format.

GeoCroissant is a geospatial extension of MLCommons' Croissant metadata standard. This enables ML-ready dataset descriptions including:

- Spatial extent (bounding box, geometry)
- Temporal coverage
- Data files (via `distribution`)
- Licensing, creators, and references

## STAC Collection to GeoCroissant Mapping
| STAC Field         | GeoCroissant Field       |
|--------------------|---------------------------|
| `title`            | `name`                   |
| `description`      | `description`            |
| `license`          | `license`                |
| `extent.spatial`   | `geocr:BoundingBox`      |
| `extent.temporal`  | `dct:temporal`           |
| `links`            | `references`             |
| `assets`           | `distribution`           |

## Import Required Libraries

In [ ]:
!pip install mlcroissant

## Load STAC Collection JSON
We load a `stac.json` file which conforms to the STAC Collection spec.
This function reads STAC fields and generates GeoCroissant metadata.

It includes handling for:
- Distribution files and MIME types
- Spatial and temporal coverage
- Title, license, creator info
- Unmapped STAC fields (for logging/debugging)

In [ ]:
import json
from datetime import datetime
import re

def sanitize_name(name):
    return re.sub(r"[^a-zA-Z0-9_\-]", "-", name)

def ensure_semver(version):
    if not version:
        return "1.0.0"
    if version.startswith("v"):
        version = version[1:]
    parts = version.split(".")
    if len(parts) == 2:
        parts.append("0")
    return ".".join(parts[:3])

def stac_to_geocroissant(stac_dict):
    dataset_id = stac_dict.get("id")
    name = sanitize_name(stac_dict.get("title", dataset_id or "UnnamedDataset"))
    version = ensure_semver(stac_dict.get("version", "1.0.0"))

    croissant = {
        "@context": {
            "@language": "en",
            "@vocab": "https://schema.org/",
            "cr": "http://mlcommons.org/croissant/",
            "geocr": "http://mlcommons.org/geocroissant/",
            "dct": "http://purl.org/dc/terms/",
            "sc": "https://schema.org/",
            "citeAs": "cr:citeAs",
            "column": "cr:column",
            "conformsTo": "dct:conformsTo",
            "data": {"@id": "cr:data", "@type": "@json"},
            "dataBiases": "cr:dataBiases",
            "dataCollection": "cr:dataCollection",
            "dataType": {"@id": "cr:dataType", "@type": "@vocab"},
            "extract": "cr:extract",
            "field": "cr:field",
            "fileProperty": "cr:fileProperty",
            "fileObject": "cr:fileObject",
            "fileSet": "cr:fileSet",
            "format": "cr:format",
            "includes": "cr:includes",
            "isLiveDataset": "cr:isLiveDataset",
            "jsonPath": "cr:jsonPath",
            "key": "cr:key",
            "md5": {"@id": "cr:md5", "@type": "sc:Text"},
            "sha256": {"@id": "cr:sha256", "@type": "sc:Text"},
            "parentField": "cr:parentField",
            "path": "cr:path",
            "personalSensitiveInformation": "cr:personalSensitiveInformation",
            "recordSet": "cr:recordSet",
            "references": "cr:references",
            "regex": "cr:regex",
            "repeated": "cr:repeated",
            "replace": "cr:replace",
            "separator": "cr:separator",
            "source": "cr:source",
            "subField": "cr:subField",
            "transform": "cr:transform"
            
        },
        "@type": "Dataset",
        "@id": dataset_id,
        "name": name,
        "description": stac_dict.get("description", ""),
        "version": version,
        "license": stac_dict.get("license", "CC-BY-4.0"),
        "conformsTo": "http://mlcommons.org/croissant/1.0"
    }

    # Citation
    if "sci:citation" in stac_dict:
        croissant["citeAs"] = stac_dict["sci:citation"]
        croissant["citation"] = stac_dict["sci:citation"]

    # Creator
    if stac_dict.get("providers"):
        provider = stac_dict["providers"][0]
        croissant["creator"] = {
            "@type": "Organization",
            "name": provider.get("name", "Unknown"),
            "url": provider.get("url", "")
        }

    # Link: self
    for link in stac_dict.get("links", []):
        if link.get("rel") == "self":
            croissant["url"] = link.get("href")
            break

    # Bounding box
    spatial = stac_dict.get("extent", {}).get("spatial", {}).get("bbox")
    if spatial:
        croissant["geocr:BoundingBox"] = spatial[0]

    # Temporal extent
    temporal = stac_dict.get("extent", {}).get("temporal", {}).get("interval")
    if temporal and temporal[0]:
        start, end = temporal[0][0], temporal[0][1]
        croissant["dct:temporal"] = {"startDate": start, "endDate": end}
        croissant["datePublished"] = start
    else:
        croissant["datePublished"] = datetime.utcnow().isoformat() + "Z"

    # Distribution - updated to pass validation
    croissant["distribution"] = []
    for key, asset in stac_dict.get("assets", {}).items():
        file_object = {
            "@type": "cr:FileObject",
            "@id": key,
            "name": key,
            "description": asset.get("description", asset.get("title", "")),
            "contentUrl": asset.get("href"),
            "encodingFormat": asset.get("type", "application/octet-stream"),
            "sha256": "https://github.com/mlcommons/croissant/issues/80",  # 64-char hex
            "md5": "https://github.com/mlcommons/croissant/issues/80"  # 32-char hex
        }
        
        # Use actual checksums if available
        if "checksum:multihash" in asset:
            file_object["sha256"] = asset["checksum:multihash"]
        elif "file:checksum" in asset:
            file_object["sha256"] = asset["file:checksum"]
        if "checksum:md5" in asset:
            file_object["md5"] = asset["checksum:md5"]
        
        croissant["distribution"].append(file_object)

    # Debug: unmapped fields
    mapped_keys = {
        "id", "type", "links", "title", "assets", "extent",
        "license", "version", "providers", "description", "sci:citation"
    }
    extra_fields = {k: v for k, v in stac_dict.items() if k not in mapped_keys}
    print("\n\033[1mUnmapped STAC Fields:\033[0m")
    if extra_fields:
        for k, v in extra_fields.items():
            print(f"- {k}: {type(v).__name__}")
    else:
        print("None ✅")

    return croissant


# Load STAC Collection JSON
with open("stac.json") as f:
    stac_data = json.load(f)

# Convert to GeoCroissant
croissant_json = stac_to_geocroissant(stac_data)

# Save GeoCroissant JSON-LD
with open("croissant.json", "w") as f:
    json.dump(croissant_json, f, indent=2)

print("\nGeoCroissant conversion complete. Output saved to 'croissant.json'")


Unmapped STAC Fields:
- renders: dict
- summaries: dict
- deprecated: bool
- item_assets: dict
- stac_version: str
- stac_extensions: list

GeoCroissant conversion complete. Output saved to 'croissant.json'


## Print Unmapped STAC Fields
Display any fields in the original STAC Collection that weren’t mapped to GeoCroissant fields — useful for debugging or future enhancements.

In [ ]:
import json

# Load and pretty-print the content of croissant.json
with open("croissant.json", "r") as f:
    croissant_data = json.load(f)

# Pretty-print JSON to console
print(json.dumps(croissant_data, indent=2))

{
  "@context": {
    "@language": "en",
    "@vocab": "https://schema.org/",
    "cr": "http://mlcommons.org/croissant/",
    "geocr": "http://mlcommons.org/geocroissant/",
    "dct": "http://purl.org/dc/terms/",
    "sc": "https://schema.org/",
    "citeAs": "cr:citeAs",
    "column": "cr:column",
    "conformsTo": "dct:conformsTo",
    "data": {
      "@id": "cr:data",
      "@type": "@json"
    },
    "dataBiases": "cr:dataBiases",
    "dataCollection": "cr:dataCollection",
    "dataType": {
      "@id": "cr:dataType",
      "@type": "@vocab"
    },
    "extract": "cr:extract",
    "field": "cr:field",
    "fileProperty": "cr:fileProperty",
    "fileObject": "cr:fileObject",
    "fileSet": "cr:fileSet",
    "format": "cr:format",
    "includes": "cr:includes",
    "isLiveDataset": "cr:isLiveDataset",
    "jsonPath": "cr:jsonPath",
    "key": "cr:key",
    "md5": {
      "@id": "cr:md5",
      "@type": "sc:Text"
    },
    "sha256": {
      "@id": "cr:sha256",
      "@type": "sc:Tex

## Convert STAC to GeoCroissant Metadata
This function maps key STAC Collection fields into GeoCroissant JSON-LD format, using predefined field correspondences.
## Run the Conversion and Output GeoCroissant
Use the conversion function to transform the STAC JSON into GeoCroissant and save it as `croissant.json`.

In [ ]:
import json
from datetime import datetime
import re


def sanitize_name(name):
    return re.sub(r"[^a-zA-Z0-9_\-]", "-", name)


def ensure_semver(version):
    if not version:
        return "1.0.0"
    if version.startswith("v"):
        version = version[1:]
    parts = version.split(".")
    if len(parts) == 2:
        parts.append("0")
    return ".".join(parts[:3])


def stac_to_geocroissant(stac_dict):
    dataset_id = stac_dict.get("id")
    name = sanitize_name(stac_dict.get("title", dataset_id or "UnnamedDataset"))
    version = ensure_semver(stac_dict.get("version", "1.0.0"))

    croissant = {
        "@context": {
            "@language": "en",
            "@vocab": "https://schema.org/",
            "cr": "http://mlcommons.org/croissant/",
            "geocr": "http://mlcommons.org/geocroissant/",
            "dct": "http://purl.org/dc/terms/",
            "sc": "https://schema.org/",
            "citeAs": "cr:citeAs",
            "column": "cr:column",
            "conformsTo": "dct:conformsTo",
            "data": {"@id": "cr:data", "@type": "@json"},
            "dataBiases": "cr:dataBiases",
            "dataCollection": "cr:dataCollection",
            "dataType": {"@id": "cr:dataType", "@type": "@vocab"},
            "extract": "cr:extract",
            "field": "cr:field",
            "fileProperty": "cr:fileProperty",
            "fileObject": "cr:fileObject",
            "fileSet": "cr:fileSet",
            "format": "cr:format",
            "includes": "cr:includes",
            "isLiveDataset": "cr:isLiveDataset",
            "jsonPath": "cr:jsonPath",
            "key": "cr:key",
            "md5": {"@id": "cr:md5", "@type": "sc:Text"},
            "sha256": {"@id": "cr:sha256", "@type": "sc:Text"},
            "parentField": "cr:parentField",
            "path": "cr:path",
            "personalSensitiveInformation": "cr:personalSensitiveInformation",
            "recordSet": "cr:recordSet",
            "references": "cr:references",
            "regex": "cr:regex",
            "repeated": "cr:repeated",
            "replace": "cr:replace",
            "separator": "cr:separator",
            "source": "cr:source",
            "subField": "cr:subField",
            "transform": "cr:transform"
        },
        "@type": "Dataset",
        "@id": dataset_id,
        "name": name,
        "description": stac_dict.get("description", ""),
        "version": version,
        "license": stac_dict.get("license", "CC-BY-4.0"),
        "conformsTo": "http://mlcommons.org/croissant/1.0"
    }

    if "sci:citation" in stac_dict:
        croissant["citeAs"] = stac_dict["sci:citation"]
        croissant["citation"] = stac_dict["sci:citation"]

    if stac_dict.get("providers"):
        provider = stac_dict["providers"][0]
        croissant["creator"] = {
            "@type": "Organization",
            "name": provider.get("name", "Unknown"),
            "url": provider.get("url", "")
        }

    # Handle 'self' URL
    for link in stac_dict.get("links", []):
        if link.get("rel") == "self":
            croissant["url"] = link.get("href")
            break

    # Handle other STAC references
    references = []
    for link in stac_dict.get("links", []):
        rel = link.get("rel")
        href = link.get("href")
        if not href or rel == "self":
            continue

        name_map = {
            "root": "STAC root catalog",
            "parent": "STAC parent catalog",
            "items": "STAC item list",
            "about": "GitHub Repository",
            "predecessor-version": "Previous version",
            "http://www.opengis.net/def/rel/ogc/1.0/queryables": "Queryables"
        }

        references.append({
            "@type": "CreativeWork",
            "url": href,
            "name": name_map.get(rel, rel),
            "encodingFormat": link.get("type", "application/json")
        })

    if references:
        croissant["references"] = references

    # Spatial and temporal extent
    spatial = stac_dict.get("extent", {}).get("spatial", {}).get("bbox")
    if spatial:
        croissant["geocr:BoundingBox"] = spatial[0]

    temporal = stac_dict.get("extent", {}).get("temporal", {}).get("interval")
    if temporal and temporal[0]:
        start, end = temporal[0][0], temporal[0][1]
        croissant["dct:temporal"] = {"startDate": start, "endDate": end}
        croissant["datePublished"] = start
    else:
        croissant["datePublished"] = datetime.utcnow().isoformat() + "Z"

    # Asset-level distribution
    croissant["distribution"] = []
    for key, asset in stac_dict.get("assets", {}).items():
        file_object = {
            "@type": "cr:FileObject",
            "@id": key,
            "name": key,
            "description": asset.get("description", asset.get("title", "")),
            "contentUrl": asset.get("href"),
            "encodingFormat": asset.get("type", "application/octet-stream"),
            "sha256": "https://github.com/mlcommons/croissant/issues/80",
            "md5": "https://github.com/mlcommons/croissant/issues/80"
        }

        if "checksum:multihash" in asset:
            file_object["sha256"] = asset["checksum:multihash"]
        elif "file:checksum" in asset:
            file_object["sha256"] = asset["file:checksum"]
        if "checksum:md5" in asset:
            file_object["md5"] = asset["checksum:md5"]

        croissant["distribution"].append(file_object)

    # item_assets as fileSet templates
    if "item_assets" in stac_dict:
        croissant["fileSet"] = []
        for key, asset in stac_dict["item_assets"].items():
            file_obj = {
                "@type": "cr:FileObject",
                "@id": key,
                "name": key,
                "description": asset.get("description", asset.get("title", "")),
                "encodingFormat": asset.get("type", "application/octet-stream"),
                "sha256": "https://github.com/mlcommons/croissant/issues/80",
                "md5": "https://github.com/mlcommons/croissant/issues/80"
            }
            file_set = {
                "@type": "cr:FileSet",
                "name": f"Template for {key}",
                "includes": [file_obj]
            }
            croissant["fileSet"].append(file_set)

    if "renders" in stac_dict:
        croissant["geocr:visualizations"] = stac_dict["renders"]

    if "summaries" in stac_dict:
        croissant["geocr:summaries"] = stac_dict["summaries"]

    if "stac_extensions" in stac_dict:
        croissant["geocr:stac_extensions"] = stac_dict["stac_extensions"]
    if "stac_version" in stac_dict:
        croissant["geocr:stac_version"] = stac_dict["stac_version"]

    if "deprecated" in stac_dict:
        croissant["isLiveDataset"] = not stac_dict["deprecated"]

    # Report unmapped fields
    mapped_keys = {
        "id", "type", "links", "title", "assets", "extent",
        "license", "version", "providers", "description", "sci:citation",
        "renders", "summaries", "stac_extensions", "stac_version", "deprecated", "item_assets"
    }
    extra_fields = {k: v for k, v in stac_dict.items() if k not in mapped_keys}
    print("\n\033[1mUnmapped STAC Fields:\033[0m")
    if extra_fields:
        for k, v in extra_fields.items():
            print(f"- {k}: {type(v).__name__}")
    else:
        print("None ")

    return croissant


# === Main Runner ===
if __name__ == "__main__":
    # Load STAC Collection JSON
    with open("stac.json") as f:
        stac_data = json.load(f)

    # Convert to GeoCroissant
    croissant_json = stac_to_geocroissant(stac_data)

    # Save GeoCroissant JSON-LD
    with open("croissant.json", "w") as f:
        json.dump(croissant_json, f, indent=2)

    print("\nGeoCroissant conversion complete. Output saved to 'croissant.json'")


Unmapped STAC Fields:
None 

GeoCroissant conversion complete. Output saved to 'croissant.json'


## Preview croissant.json
Use this cell to inspect the structure and values of the converted metadata.

In [ ]:
import json

# Load and pretty-print the content of croissant.json
with open("croissant.json", "r") as f:
    croissant_data = json.load(f)

# Pretty-print JSON to console
print(json.dumps(croissant_data, indent=2))

{
  "@context": {
    "@language": "en",
    "@vocab": "https://schema.org/",
    "cr": "http://mlcommons.org/croissant/",
    "geocr": "http://mlcommons.org/geocroissant/",
    "dct": "http://purl.org/dc/terms/",
    "sc": "https://schema.org/",
    "citeAs": "cr:citeAs",
    "column": "cr:column",
    "conformsTo": "dct:conformsTo",
    "data": {
      "@id": "cr:data",
      "@type": "@json"
    },
    "dataBiases": "cr:dataBiases",
    "dataCollection": "cr:dataCollection",
    "dataType": {
      "@id": "cr:dataType",
      "@type": "@vocab"
    },
    "extract": "cr:extract",
    "field": "cr:field",
    "fileProperty": "cr:fileProperty",
    "fileObject": "cr:fileObject",
    "fileSet": "cr:fileSet",
    "format": "cr:format",
    "includes": "cr:includes",
    "isLiveDataset": "cr:isLiveDataset",
    "jsonPath": "cr:jsonPath",
    "key": "cr:key",
    "md5": {
      "@id": "cr:md5",
      "@type": "sc:Text"
    },
    "sha256": {
      "@id": "cr:sha256",
      "@type": "sc:Tex

# Validate with mlcroissant
We use the `mlcroissant` CLI to validate the structure of the generated JSON-LD.
```bash
!mlcroissant validate --jsonld=croissant.json
```

In [17]:
!mlcroissant validate --jsonld=croissant.json

W0629 17:30:03.355020 138358395765120 rdf.py:80] WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The different keys are: {'examples', 'rai'}
I0629 17:30:03.372449 138358395765120 validate.py:53] Done.


In [18]:
from mlcroissant import Dataset

# Load dataset
dataset = Dataset("croissant.json")

# Print validation issues (errors + warnings)
print("\nDataset loaded successfully.")
print("Validation Report:-")

# Access and report issues correctly from the context
issues = dataset.metadata.ctx.issues

# Print errors
if issues.errors:
    print("\nErrors:")
    for error in issues.errors:
        print(f"  - {error}")
else:
    print("\nNo errors found.")

# Print warnings
if issues.warnings:
    print("\nWarnings:")
    for warning in issues.warnings:
        print(f"  - {warning}")
else:
    print("\nNo warnings found.")


Dataset loaded successfully.
Validation Report:-

No errors found.

No warnings found.


## Print Reference Links
This cell prints external references linked via the `references` property in GeoCroissant.

In [19]:
import json

# Load your croissant.json
with open("croissant.json", "r") as f:
    croissant = json.load(f)

# Print all reference links
print("\nReferenced STAC Links:")
for ref in croissant.get("references", []):
    name = ref.get("name")
    url = ref.get("url")
    fmt = ref.get("encodingFormat")
    print(f"  - {name}: {url} ({fmt})")


Referenced STAC Links:
  - STAC item list: https://stac.maap-project.org/collections/icesat2-boreal-v2.1-agb/items (application/geo+json)
  - STAC parent catalog: https://stac.maap-project.org/ (application/json)
  - STAC root catalog: https://stac.maap-project.org/ (application/json)
  - GitHub Repository: https://github.com/lauraduncanson/icesat2_boreal (text/html)
  - Previous version: https://stac.maap-project.org/collections/icesat2-boreal (application/json)
  - Queryables: https://stac.maap-project.org/collections/icesat2-boreal-v2.1-agb/queryables (application/schema+json)


## Print STAC Items & Distribution URLs
We extract and list all STAC item links and distribution file download URLs from `croissant.json`.

In [20]:
import json

with open("croissant.json") as f:
    data = json.load(f)

# STAC item list from base collection URL
url = data.get("url", "")
if url:
    print(f"- STAC item list: {url.rstrip('/')}/items")

# STAC reference map
ref_map = {
    "parent": "STAC parent catalog",
    "root": "STAC root catalog",
    "github": "GitHub Repository",
    "previous": "Previous version",
    "queryables": "Queryables"
}

for r in data.get("references", []):
    for key, label in ref_map.items():
        if key in r.get("name", "").lower():
            print(f"  - {label}: {r['url']} ({r.get('encodingFormat', 'application/json')})")

# Distributions
dists = data.get("distribution", [])
if dists:
    print("\nReferenced Distribution Files:")
    for d in dists:
        print(f"  - {d.get('name', 'Unnamed')} ({d.get('encodingFormat', 'unknown')}): {d.get('contentUrl', 'No URL')}")

- STAC item list: https://stac.maap-project.org/collections/icesat2-boreal-v2.1-agb/items
  - STAC parent catalog: https://stac.maap-project.org/ (application/json)
  - STAC root catalog: https://stac.maap-project.org/ (application/json)
  - GitHub Repository: https://github.com/lauraduncanson/icesat2_boreal (text/html)
  - Previous version: https://stac.maap-project.org/collections/icesat2-boreal (application/json)
  - Queryables: https://stac.maap-project.org/collections/icesat2-boreal-v2.1-agb/queryables (application/schema+json)

Referenced Distribution Files:
  - tiles (application/geopackage+sqlite3): s3://nasa-maap-data-store/file-staging/nasa-map/icesat2-boreal-v2.1/agb/boreal_tiles_v004_AGB_H30_2020_ORNLDAAC.gpkg


In [21]:
import requests

# Load croissant.json
with open("croissant.json", "r") as f:
    croissant = json.load(f)

# Find STAC item list URL from references
item_list_url = None
for ref in croissant.get("references", []):
    if "item list" in ref.get("name", "").lower():
        item_list_url = ref.get("url")
        break

if item_list_url:
    print("\nSTAC Items:")
    stac_items = requests.get(item_list_url).json()
    for feature in stac_items.get("features", []):
        item_id = feature.get("id")
        date = feature.get("properties", {}).get("datetime", "")
        print(f"- {item_id} | {date}")
        for asset_key, asset in feature.get("assets", {}).items():
            print(f"   {asset_key.upper()}: {asset['href']}")
else:
    print("STAC item list URL not found in references.")


STAC Items:
- boreal_agb_2020_202501211737487322_0039261 | 2020-07-01T23:59:59.500000Z
   COG: s3://nasa-maap-data-store/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0039261/boreal_agb_2020_202501211737487322_0039261.tif
   TRAINING_DATA_CSV: s3://nasa-maap-data-store/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0039261/boreal_agb_2020_202501211737487322_0039261_train_data.csv
- boreal_agb_2020_202411261732638547_0001981 | 2020-07-01T23:59:59.500000Z
   COG: s3://nasa-maap-data-store/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0001981/boreal_agb_2020_202411261732638547_0001981.tif
   TRAINING_DATA_CSV: s3://nasa-maap-data-store/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0001981/boreal_agb_2020_202411261732638547_0001981_train_data.csv
- boreal_agb_2020_202411261732638346_0003509 | 2020-07-01T23:59:59.500000Z
   COG: s3://nasa-maap-data-store/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0003509/boreal_agb_2020_202411261732638346_0003509.tif
   TRAINING_DATA_CSV: s3://nasa-maap-

In [22]:
import rasterio
import json
import numpy as np

# URL of the COG file via /vsicurl
cog_url = "/vsicurl/https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0039261/boreal_agb_2020_202501211737487322_0039261.tif"

with rasterio.Env(AWS_NO_SIGN_REQUEST="YES"):
    with rasterio.open(cog_url) as src:
        # ---- Print metadata (JSON safe) ----
        meta = src.meta.copy()
        if 'crs' in meta and meta['crs'] is not None:
            meta['crs'] = meta['crs'].to_string()

        print("Metadata:")
        print(json.dumps(meta, indent=2))

        # ---- Print image summary ----
        print(f"\nCRS: {src.crs}")
        print(f"Shape: {src.width}x{src.height}, Bands: {src.count}")

        for i in range(1, src.count + 1):
            band = src.read(i, masked=True)  # Returns a MaskedArray
            mask = band.mask
            data = band.data

            total_pixels = band.size
            valid_pixels = np.count_nonzero(~mask)
            nan_pixels = np.count_nonzero(mask)

            print(f"\nBand {i} Stats:")
            print(f"  - Shape         : {band.shape}")
            print(f"  - Total pixels  : {total_pixels}")
            print(f"  - Valid pixels  : {valid_pixels}")
            print(f"  - NaNs (masked) : {nan_pixels}")

            if valid_pixels > 0:
                print(f"  - Min           : {data[~mask].min():.6f}")
                print(f"  - Max           : {data[~mask].max():.6f}")
                print(f"  - Mean          : {data[~mask].mean():.6f}")
            else:
                print("No valid data in this band.")

Metadata:
{
  "driver": "GTiff",
  "dtype": "float32",
  "nodata": NaN,
  "width": 3000,
  "height": 3000,
  "count": 2,
  "crs": "PROJCS[\"unnamed\",GEOGCS[\"GRS 1980(IUGG, 1980)\",DATUM[\"unknown\",SPHEROID[\"GRS80\",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM[\"Greenwich\",0],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]]],PROJECTION[\"Albers_Conic_Equal_Area\"],PARAMETER[\"latitude_of_center\",40],PARAMETER[\"longitude_of_center\",180],PARAMETER[\"standard_parallel_1\",50],PARAMETER[\"standard_parallel_2\",70],PARAMETER[\"false_easting\",0],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH]]",
  "transform": [
    30.0,
    0.0,
    2078521.9999999953,
    0.0,
    -30.0,
    4383304.000000009,
    0.0,
    0.0,
    1.0
  ]
}

CRS: PROJCS["unnamed",GEOGCS["GRS 1980(IUGG, 1980)",DATUM["unknown",SPHEROID["GRS80",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],U

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
import numpy as np
from PIL import Image
import io
import base64
import folium

def reproject_to_wgs84(src_url):
    with rasterio.open(src_url) as src:
        dst_crs = 'EPSG:4326'
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        memfile = MemoryFile()
        with memfile.open(**kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
        return memfile.open()

def image_to_base64_png(dataset, band_index):
    band = dataset.read(band_index)
    norm = np.clip((band - np.nanmin(band)) / (np.nanpercentile(band, 98) - np.nanmin(band)), 0, 1)
    rgb = np.uint8(norm * 255)
    rgb = np.stack([rgb] * 3, axis=-1)
    
    img = Image.fromarray(rgb)
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

# URL to your raster
cog_url = "https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0039261/boreal_agb_2020_202501211737487322_0039261.tif"

# Reproject
reprojected_ds = reproject_to_wgs84(cog_url)
bounds = reprojected_ds.bounds

# Create Folium map
m = folium.Map(
    location=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2],
    zoom_start=6,
    tiles='cartodbpositron'
)

# Add each band as overlay
for band_index in range(1, reprojected_ds.count + 1):
    img_base64 = image_to_base64_png(reprojected_ds, band_index)
    folium.raster_layers.ImageOverlay(
        image=f"data:image/png;base64,{img_base64}",
        bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
        opacity=0.6,
        name=f"Band {band_index}"
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

## Raster + Vector + CSV Preview using Folium
This optional section demonstrates how to:

- Reproject a COG TIFF to EPSG:4326
- Display with Folium
- Add vector (GeoPackage) and point (CSV) layers

Useful for visual QA of spatial coverage.

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import requests
from folium.plugins import MarkerCluster
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
import numpy as np
from PIL import Image
import io
import base64

# --- Function to reproject raster to WGS84 ---
def reproject_to_wgs84(src_url):
    with rasterio.open(src_url) as src:
        dst_crs = 'EPSG:4326'
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        memfile = MemoryFile()
        with memfile.open(**kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
        return memfile.open()

# --- Convert a raster band to base64 PNG for folium overlay ---
def image_to_base64_png(dataset, band_index):
    band = dataset.read(band_index)
    norm = np.clip((band - np.nanmin(band)) / (np.nanpercentile(band, 98) - np.nanmin(band)), 0, 1)
    rgb = np.uint8(norm * 255)
    rgb = np.stack([rgb] * 3, axis=-1)
    img = Image.fromarray(rgb)
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

# --- Initialize base map ---
m = folium.Map(location=[60, -100], zoom_start=4, tiles="CartoDB positron")

# --- 1. AGB COG Image Overlay ---
cog_url = "https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0039261/boreal_agb_2020_202501211737487322_0039261.tif"
reprojected_ds = reproject_to_wgs84(cog_url)
bounds = reprojected_ds.bounds
img_base64 = image_to_base64_png(reprojected_ds, 1)

folium.raster_layers.ImageOverlay(
    image=f"data:image/png;base64,{img_base64}",
    bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
    opacity=0.6,
    name="AGB COG Image (Band 1)"
).add_to(m)

# --- 2. GPKG Tile Boundaries ---
gpkg_url = "https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/icesat2-boreal-v2.1/agb/boreal_tiles_v004_AGB_H30_2020_ORNLDAAC.gpkg"
gpkg_path = "boreal_tiles.gpkg"
with open(gpkg_path, "wb") as f:
    f.write(requests.get(gpkg_url).content)

gdf = gpd.read_file(gpkg_path).to_crs("EPSG:4326")
tooltip = folium.GeoJsonTooltip(fields=["tile_id"]) if "tile_id" in gdf.columns else None

folium.GeoJson(
    gdf,
    name="Tile Boundaries",
    tooltip=tooltip
).add_to(m)

# --- 3. Training Data Points ---
csv_url = "https://nasa-maap-data-store.s3.amazonaws.com/file-staging/nasa-map/icesat2-boreal-v2.1/agb/0039261/boreal_agb_2020_202501211737487322_0039261_train_data.csv"
csv_path = "train_data.csv"
with open(csv_path, "wb") as f:
    f.write(requests.get(csv_url).content)

df = pd.read_csv(csv_path)

marker_cluster = MarkerCluster(name="Training Data").add_to(m)

# Fixed lat/lon column names and popup text
for _, row in df.iterrows():
    lat, lon = row.get("lat"), row.get("lon")
    if pd.notna(lat) and pd.notna(lon):
        popup_text = f"AGB: {row.get('AGB', 'NA')} ± {row.get('SE', 'NA')}"
        folium.CircleMarker(
            location=[lat, lon],
            radius=2,
            color="blue",
            fill=True,
            fill_opacity=0.6,
            popup=popup_text
        ).add_to(marker_cluster)

# --- Add layer control ---
folium.LayerControl().add_to(m)

# --- Display inline in Jupyter ---
m